# ライン情報と選手の脚質を取得する

## 取得元: WinTicket

In [115]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [116]:
racecard_url = "https://www.winticket.jp/keirin/yokkaichi/racecard/2022010448/3/8"
raceresult_url = "https://www.winticket.jp/keirin/yokkaichi/raceresult/2022010448/3/8"
req = requests.get(raceresult_url)
soup = BeautifulSoup(req.content, 'html.parser')

### 日程を与えるとその日開催されたレースのURLをリストで返す関数

In [117]:
def get_urls(year, month, day):
    url = "https://www.winticket.jp/keirin/racecard/" + str(year) + str(month).zfill(2) + str(day).zfill(2)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    url_soup = soup.find_all('a', rel='nofollow')
    urls = [i.get('href') for i in url_soup]
    return urls

### htmlを与えるとライン情報のデータフレームを返す関数

In [118]:
def create_line_table(html):
    line_soup = html.find_all("div", class_="sc-1y958x7-0")
    lines = [i.text for i in line_soup][0].split('区切り')

    line_data = [[0 for j in range(7)] for i in range(3)]
    for n, i in enumerate(lines):
        people = len(i)
        for k, j in enumerate(i):
            index = int(j) - 1
            line_data[0][index] = people
            line_data[1][index] = n
            line_data[2][index] = k
    df = pd.DataFrame({'people':line_data[0],
                       'group':line_data[1],
                       'number':line_data[2]})
    return df

### htmlを与えると着順のデータフレームを返す関数

In [119]:
def create_result_table(html):
    result_soup = html.find_all("div", class_="sc-eo68xd-0")
    result = [i.text for i in result_soup]
    df = pd.DataFrame({'result':result})
    return df

In [124]:
race_tables = pd.read_html(racecard_url)
race_table = race_tables[1]
legtype_table = race_table[['車', '脚']].rename(columns={'車':'no', '脚':'leg'}).droplevel(1, axis=1)
legtype_table

,no,leg
0,1,両
1,2,両
2,3,両
3,4,追
4,5,逃
5,6,逃
6,7,追
